In [1]:
import pandas as pd
import os
import numpy as np
from pycox.evaluation.concordance import concordance_td
from pycox.evaluation import EvalSurv

In [2]:
os.getcwd()

'/home/ronin/Dev/miniconda3/notebooks/thesis_notebook'

In [3]:
#pth = "zenodo/metrics/TCGA/BLCA/early/clinical_gex/"
#pthsf = "zenodo/survival_functions/TCGA/BLCA/early/clinical_gex/"
metrics_csv = "metrics/BLCA/intermediate_mean/clinical_gex/metrics.csv"
data_csv = "processed/BLCA_data_preprocessed.csv"
splits_test_csv = "splits/BLCA_test_splits.csv"
splits_train_csv = "splits/BLCA_train_splits.csv"
survival_funct_path = "survival_functions/BLCA/late_mean/clinical_gex/" ## + split_1.csv etc

In [4]:
#split = pd.read_csv(pthsf + "split_1.csv")
splits_test = pd.read_csv(splits_test_csv)
splits_test.shape

(25, 66)

In [5]:
X = pd.read_csv(data_csv)

In [6]:
event_indicator, event_time = X["OS"].to_numpy(), X["OS_days"].to_numpy()

In [9]:
pd.read_csv(survival_funct_path + f"split_1.csv").to_numpy().shape

(66, 243)

In [10]:
l = []
split = pd.read_csv(splits_test_csv)

for idx in range(25):
    surv = pd.read_csv(survival_funct_path + f"split_{idx+1}.csv").to_numpy().T
    test_idx = split.iloc[idx, :].tolist()
    
    cidx = concordance_td(
        method='adj_antolini',
        durations = event_time[test_idx],
        events = event_indicator[test_idx],
        surv = surv,
        surv_idx= np.searchsorted(np.unique(test_idx), 
                                  test_idx)
    )
    l.append(cidx)
    # unique già fa il sorting

In [11]:
sum(pd.read_csv("metrics/BLCA/late_mean/clinical_gex/metrics.csv")["concordance"] - np.array(l))/ len(l)

0.005347292964327583

In [12]:
((pd.read_csv("metrics/BLCA/late_mean/clinical_gex/metrics.csv")["concordance"] - l)/len(l))

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000470
5     0.000260
6     0.000000
7     0.000570
8     0.000000
9     0.000000
10    0.000220
11    0.000000
12    0.000681
13    0.000504
14    0.000000
15    0.000660
16    0.000000
17    0.000000
18    0.000000
19    0.000075
20    0.000000
21    0.000000
22    0.001608
23    0.000000
24    0.000298
Name: concordance, dtype: float64

In [ ]:
## approccio per trovare time_idx
#for i, time in enumerate(event_time):
 #   result[i] = np.where(unique_times == time)[0][0]

In [18]:
np.array(l).mean()

0.6081278246625585

In [30]:
(pd.read_csv("metrics/BLCA/late_mean/clinical_gex/metrics.csv")['concordance'])

0     0.658065
1     0.677479
2     0.507077
3     0.556391
4     0.543103
5     0.639757
6     0.556650
7     0.679343
8     0.613742
9     0.597893
10    0.662997
11    0.564199
12    0.586022
13    0.630000
14    0.641716
15    0.643788
16    0.602687
17    0.683753
18    0.630538
19    0.588071
20    0.640040
21    0.514163
22    0.600082
23    0.618182
24    0.701139
Name: concordance, dtype: float64

In [50]:
pd.read_csv(survival_funct_path+"split_1.csv").shape

(66, 243)

In [ ]:
### 
### subordination theory 
###

In [20]:
rdf = pd.DataFrame(columns=['StringColumn', 'StringColumn', 'ValueColumn'])

# Add a new row with a string and a value
rdf.loc[0] = ['Cancer', 'Model' , np.array(l).mean()]

# Print the DataFrame
print(rdf)

  StringColumn StringColumn  ValueColumn
0       Cancer        Model     0.608128


In [22]:
X.shape

(330, 84758)